In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc
import hvplot.pandas

### Pre-Eruption Period

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F11.pkl', 'rb') as F:
    botpt_data = pk.load(F)
df_botptF = pd.DataFrame(botpt_data)
df_botptF['bottom_pressure'] = df_botptF['bottom_pressure'].astype(float)
df_botptF['depth']=df_botptF['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptF.index.to_pydatetime()]
df_botptF['epoch'] = epoch
df_botptF= df_botptF.sort_index()

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_E11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE= df_botptE.sort_index()

#### Cleaning up the DataFrame

In [ ]:
del df_botptF['epoch']
del df_botptF['bottom_pressure']
del df_botptE['epoch']
del df_botptE['bottom_pressure']

#### Merge BOTPT E and BOTPT F 

In [ ]:
test = pd.merge(df_botptF, df_botptE,how='outer', indicator=True, left_index=True, right_index=True, suffixes=('_F', '_E'))

In [ ]:
df_botptMerge = test[test['_merge'] == 'both']
del df_botptMerge['_merge']

In [ ]:
df_botptMerge = df_botptMerge.loc['2014-01-01 00:00:00':'2015-05-31 00:00:00']

#### Calculate Depth difference 

In [ ]:
depthDiff = df_botptMerge['depth_E'].values - df_botptMerge['depth_F'].values

In [ ]:
df_botptMerge['diff'] = depthDiff

#### Create time and height vectors for plotting 

In [ ]:
df_botptMerge['date']=pd.DatetimeIndex(df_botptMerge.index).date

In [ ]:
df_botptMean=df_botptMerge.groupby('date').mean()
# df_botptMean.head

#### Plot One Day Measurements. 

In [ ]:
df_botptMean.hvplot(x='date', y='diff')
# df_botptMean.hvplot(x='date', y='diff', datashade=True)
# hvplot.save(Examples, 'test.png')
# df_subsetBOTPT['dtide'].hvplot(x='epoch', y='dtide', datashade=True)
# df_subsetBOTPT['dtide'].hvplot.line(x='epoch', y='detided depth')

### Remove Spikes  

In [ ]:
weird_data = np.array(height)
time_array = np.array(time)

cleaner_data = []
cleaner_time = []

# i = 0
# while i < len(weird_data):
#     if (weird_data[i] - weird_data[i-1] > 2

length = len(weird_data)

i = 0
while i < length:
    j = i+2
    while j < length:
        if np.abs(weird_data[i] - weird_data[j]) < .05:
            cleaner_data.append(weird_data[j])
            cleaner_time.append(time_array[j])
            break
        else:
            j += 1
    i = j


#### Import Seismic Data 

In [ ]:
seismic_file = '/home/jovyan/data/hypo71_2018.dat'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2018-10-31 00:00:00']
del df_seismic_data['yyyymmdd']
del df_seismic_data['HHMM']
del df_seismic_data['SSS.SS']
del df_seismic_data['MW']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
# df_seismic_data.head()

#### Create dataframe with Earthquake frequency (count per day)
##### Note: Days with zero earthquakes are not represented in this timeseries. 

In [ ]:
df_eqMean=df_seismic_data.groupby('date').count()
del df_eqMean['Lat(D']
del df_eqMean['M)']
del df_eqMean['Lon(D']
del df_eqMean['M).1']
del df_eqMean['Depth']
del df_eqMean['RMS']
del df_eqMean['PMom']
del df_eqMean['SMom']
df_eqMean['count']= df_eqMean.datetime
del df_eqMean['datetime']
df_eqMean.columns.name = df_eqMean.index.name
df_eqMean.index.name = None
df_eqMean.head()

#### Resample. Add days to our earthquake data where there were zero earthquakes. 

In [ ]:
idx = pd.date_range('2017-01-1 00:00:00', '2018-10-31 00:00:00')

s = df_eqMean

s.index = pd.DatetimeIndex(s.index)

s = s.reindex(idx, fill_value=0)

In [ ]:
count = df_eqMean['count']
count.head

In [ ]:
df_botptMean.hvplot(x='date', y='diff')
# df_eqMean.hvplot(x='date', y='count')
# count.hvplot

In [ ]:
df_eqMean.hvplot(x='date', y='count')

### Three-month Intervals

#### 1. September 2014 - November 2014

In [ ]:
df_botptMergeB = df_botptMerge.loc['2014-09-01 00:00:00':'2014-11-01 00:00:00']

In [ ]:
depthDiffB = df_botptMergeB['depth_E'].values - df_botptMergeB['depth_F'].values

In [ ]:
df_botptMergeB['diff'] = depthDiffB

In [ ]:
df_botptMergeB['date']=pd.DatetimeIndex(df_botptMergeB.index).date

In [ ]:
df_botptMeanB=df_botptMergeB.groupby('date').mean()
# df_botptMean.head

In [ ]:
# df_botptMeanB.hvplot(x='date', y='diff')

In [ ]:
idxB = pd.date_range('2014-09-01 00:00:00', '2014-11-30 00:00:00')

s = df_eqMean

s.index = pd.DatetimeIndex(s.index)

s = s.reindex(idx, fill_value=0)

In [ ]:
countB = df_eqMean['count']
# count.head

In [ ]:
df_botptMeanB.hvplot(x='date', y='diff')

In [ ]:
df_eqMean.hvplot(x='date', y='count')

#### 2. November 2014 - February 2015

In [ ]:
df_botptMergeC= df_botptMerge.loc['2014-11-01 00:00:00':'2015-02-01 00:00:00']

In [ ]:
depthDiffC = df_botptMergeC['depth_E'].values - df_botptMergeC['depth_F'].values

In [ ]:
df_botptMergeC['diff'] = depthDiffC

In [ ]:
df_botptMergeC['date']=pd.DatetimeIndex(df_botptMergeC.index).date

In [ ]:
df_botptMeanC=df_botptMergeC.groupby('date').mean()

In [ ]:
df_botptMeanC.hvplot(x='date', y='diff')

#### 3. February 2015 - May 2015

In [ ]:
df_botptMergeD= df_botptMerge.loc['2015-02-01 00:00:00':'2015-05-01 00:00:00']

In [ ]:
depthDiffD = df_botptMergeD['depth_E'].values - df_botptMergeD['depth_F'].values

In [ ]:
df_botptMergeD['diff'] = depthDiffD

In [ ]:
df_botptMergeD['date']=pd.DatetimeIndex(df_botptMergeD.index).date

In [ ]:
df_botptMeanD=df_botptMergeD.groupby('date').mean()

In [ ]:
df_botptMeanD.hvplot(x='date', y='diff')